In [1]:
##IMPORTING IMPORTANT LIBRARIES
import pandas as pd
import re
from nltk.corpuss import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
import itertools
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB

In [61]:
dataset = pd.read_csv('train.csv')

In [58]:
dataset = dataset.dropna() ##Droping nan values
dataset.reset_index(inplace=True) ##Resetting indexx
messages = dataset.copy() ##creating a copy

In [60]:
## Geting the independent features from the dataset
X = dataset.drop(['index','id','author','text','label'], axis = 1)

In [49]:
X['title'][3]

'15 Civilians Killed In Single US Airstrike Have Been Identified'

In [55]:
y = dataset['label']

In [6]:
ps = PorterStemmer() ##Making Porter Stemmer object
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i]) ##Removing everything except a-z and A-Z and special characters
    review = review.lower() ## Converting title to lower
    review = review.split() ##  Spliting every word as a list's element
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')] ##Removing stop words i.e is ,or ,and etc
    review = ' '.join(review) ## joining every word back into sentence
    corpus.append(review) ## adding text to corpus

In [7]:
corpus[3]

'civilian kill singl us airstrik identifi'

In [8]:
X['title'][3]

'15 Civilians Killed In Single US Airstrike Have Been Identified'

In [9]:
cv = CountVectorizer(max _features = 5000, ngram_range=(1,3)) ##Making counter vectorizer object
bag = cv.fit_transform(corpus).toarray() ## applying the cv algorithm to make bag of words
bag.shape

(18285, 5000)

In [47]:
## Dividing Dataset into train and test 
X_train, X_test, y_train, y_test = train_test_split(bag, y, test_size = 0.3, random_state = 0)

In [52]:
# linear_clf = PassiveAggressiveClassifier(max_iter=50)

In [53]:
X_test[3]

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [68]:
classifier=MultinomialNB(alpha=0.7) ##Making classifier object
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
score = metrics.accuracy_score(y_test, y_pred)
score

0.9035727305869486

In [64]:
previous_score=0
for alpha in np.arange(0,1,0.1): ## GETIING THE BEST ALPHA VALUES FOR THE ALGORITHM
    sub_classifier=MultinomialNB(alpha=alpha)
    sub_classifier.fit(X_train,y_train)
    y_pred=sub_classifier.predict(X_test)
    score = metrics.accuracy_score(y_test, y_pred)
    if score>previous_score:
        classifier=sub_classifier
    print("Alpha: {}, Score : {}".format(alpha,score*100))
classifier

s:\python\lib\site-packages\sklearn\naive_bayes.py:506: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn('alpha too small will result in numeric errors, '


Alpha: 0.0, Score : 89.04484141450966
Alpha: 0.1, Score : 90.30258840685381
Alpha: 0.2, Score : 90.30258840685381
Alpha: 0.30000000000000004, Score : 90.26613197229311
Alpha: 0.4, Score : 90.19321910317171
Alpha: 0.5, Score : 90.24790375501276
Alpha: 0.6000000000000001, Score : 90.28436018957346
Alpha: 0.7000000000000001, Score : 90.35727305869486
Alpha: 0.8, Score : 90.30258840685381
Alpha: 0.9, Score : 90.32081662413415


MultinomialNB(alpha=0.9, class_prior=None, fit_prior=True)

In [69]:
import pickle
pickle_out = open('MultiNomial.pickle', 'wb')
pickle.dump(classifier, pickle_out)
pickle_out.close()

## TESTING



In [23]:
string = ['mandfvdvfd eats girlfriends booty for the first time dies from E. coil']

In [24]:
test=cv.transform(string)


In [35]:
test.shape

(1, 5000)

In [36]:
type(test)


scipy.sparse.csr.csr_matrix

In [37]:
testnp = np.array(test)
testnp

array(<1x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 3 stored elements in Compressed Sparse Row format>, dtype=object)

In [41]:
classifier.predict_proba(test)[0][1]

0.10724620243978136

In [43]:
classifier.classes_

array([0, 1], dtype=int64)

In [33]:
new_classifier = MultinomialNB(alpha = 0.1)

In [34]:
new_model = new_classifier.fit(bag,y)

In [70]:
testset = pd.read_csv('test.csv')
testset


,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...
...,...,...,...,...
5195,25995,The Bangladeshi Traffic Jam That Never Ends - ...,Jody Rosen,Of all the dysfunctions that plague the world’...
5196,25996,John Kasich Signs One Abortion Bill in Ohio bu...,Sheryl Gay Stolberg,WASHINGTON — Gov. John Kasich of Ohio on Tu...
5197,25997,"California Today: What, Exactly, Is in Your Su...",Mike McPhate,Good morning. (Want to get California Today by...
5198,25998,300 US Marines To Be Deployed To Russian Borde...,NaN,« Previous - Next » 300 US Marines To Be Deplo...


In [71]:
testset = testset.dropna()
testset

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...
6,20806,Pelosi Calls for FBI Investigation to Find Out...,Pam Key,"Sunday on NBC’s “Meet the Press,” House Minori..."
...,...,...,...,...
5194,25994,Trump on If ’Tapes’ Exist of Comey Conversatio...,Pam Key,Pres. Trump on if “tapes” exist of his convers...
5195,25995,The Bangladeshi Traffic Jam That Never Ends - ...,Jody Rosen,Of all the dysfunctions that plague the world’...
5196,25996,John Kasich Signs One Abortion Bill in Ohio bu...,Sheryl Gay Stolberg,WASHINGTON — Gov. John Kasich of Ohio on Tu...
5197,25997,"California Today: What, Exactly, Is in Your Su...",Mike McPhate,Good morning. (Want to get California Today by...


In [30]:
testset = testset.drop(['id','author','text'], axis = 1)

NameError: name 'testset' is not defined

In [ ]:
testset

In [31]:
testset = cv.fit_transform(testset)

NameError: name 'testset' is not defined

In [ ]:
testset[0]

In [ ]:
new_model.predict(testset)